In [129]:
# Necessary imports
import sys
import urllib.request
from requests import request
import numpy as np
import pandas as pd
import json

import re
from bs4 import BeautifulSoup
import lxml


# Save a reference to the original
# standard output
original_stdout = sys.stdout
  
# as an example, taken my article list
# published link page and stored in local
with urllib.request.urlopen('https://inventory.raw.pm/tools.html') as webPageResponse:
    outputHtml = webPageResponse.read()
  
soup = BeautifulSoup(outputHtml, 'html.parser')
# Scraped contents are placed in 
# samplehtml.html file and getting
# used for next set of examples
#with open('samplehtml.html', 'w') as f:
      
    # Here the  standard output is 
    # written to the file that we 
    # used above
    #sys.stdout = f
    #print(outputHtml)
      
    # Reset the standard output to its 
    # original value
    #sys.stdout = original_stdout
    
token = open("access_token",'r').read()
root_dir = open("root_dir",'r').read()

In [7]:
### Category Extraction
test = soup.find_all('div')
toc = test[13]

categories = []
subc = toc.find_all('a')
for item in subc[:24]:
    categories.append(item.text)
print(categories)
print(len(categories))

['Adversary Simulation', 'Binary Exploitation', 'Bug Bounty', 'Cloud', 'Code Analysis', 'Collaboration and Report', 'Configuration Audit', 'Cracking', 'Crisis Management', 'Cryptography', 'Defensive', 'Digital Forensics', 'Hardware', 'Honeypot and Decoy', 'Incident Response', 'Intentionally Vulnerable Applications', 'Networking', 'OSINT and Reconnaissance', 'Other', 'Plugins', 'Red Teaming', 'Reverse Engineering', 'Steganography', 'System Exploitation']
24


In [8]:
df = pd.DataFrame(columns=['name', 'description', 'git_url','prog_lang','category'])

In [9]:
### Table extraction
tables = soup.find_all("tbody")

for j,table in enumerate(tables):
    
    #print("\n%s:"%categories[j])
    lines = table.find_all('td')
    for i,line in enumerate(lines):
        try:
            git_url = line.select('.source-link')[0].get("href")
            name = lines[i-2].text
            description = lines[i+1].text
            prog_language = lines[i+2].text
            row = [name,description,git_url,prog_language,categories[j]]
            #print(row)
            df.loc[len(df)] = row
            #print(",".join())
        except:
            continue

In [10]:
df.head()

,name,description,git_url,prog_lang,category
0,Adversary Emulation Library,A library of adversary emulation plans to allo...,https://github.com/center-for-threat-informed-...,C,Adversary Simulation
1,Atomic Red Team,A library of tests mapped to the MITRE ATT&CK®...,https://github.com/redcanaryco/atomic-red-team,Powershell,Adversary Simulation
2,Caldera,Cyber security platform designed to easily aut...,https://github.com/mitre/caldera,Python,Adversary Simulation
3,Invoke-Apex,PowerShell-based toolkit consisting of a colle...,https://github.com/secmode/Invoke-Apex,PowerShell,Adversary Simulation
4,Manticore,Adversary emulation command line tool is parsi...,https://github.com/Manticore-Platform/manticor...,Go,Adversary Simulation


In [12]:
df[df['git_url']=='https://github.com/MISP/'] = df[df['git_url']=='https://github.com/MISP/'].replace(to_replace='https://github.com/MISP/', value='https://github.com/MISP')


In [13]:
df[df['git_url']=='https://github.com/MISP']

,name,description,git_url,prog_lang,category
57,MISP,"Malware Information Sharing Platform, an Open ...",https://github.com/MISP,PHP,Collaboration and Report


In [14]:
df[df['git_url']=='https://github.com/lucasmartinelle/Bounty-Dashboard']

,name,description,git_url,prog_lang,category


In [260]:
# Retrieve Repo Descriptions
for i in [*range(0,df.shape[0])]:
    repo_id = df[df.index==i]['git_url'].values[0][19:]
    git_api_url = "https://api.github.com/repos/" + repo_id
    #print(git_api_url)
    res = request('GET',url=git_api_url,auth=('wazaahhh', token))
    if res.status_code == 200 and len(res.text)>2 :
        #k=0
        J = res.json()
        f = open(root_dir + "data/" + re.sub("/","_",repo_id) + "_description.json","w")
        f.write(json.dumps(J))
        f.close()

        print(i,repo_id)
        #for j in J:
        #    if len(j['assets']) >0 :
        #        k+=1
        #print(k)

0 cryptolok/ASLRay
1 angr/heaphopper
2 hellman/libformatstr
3 Gallopsled/pwntools
4 peter50216/pwntools-ruby
5 JonathanSalwan/ROPgadget
6 codingo/bbr
7 serain/bbrecon
8 gwen001/BBstats
9 lucasmartinelle/Bounty-Dashboard
10 arkadiyt/bounty-targets
11 avlidienbrunn/bountydash
12 fransr/bountyplz
13 sokaRepo/bugbounty
14 infosec-au/bugbountydash
15 oreoshake/hackerone-client
16 humblelad/Needle
17 VirtueSecurity/aws-extender-cli
18 RhinoSecurityLabs/pacu
19 BishopFox/smogcloud
20 abhi-r3v0/Adhrit
21 AndroBugs/AndroBugs_Framework
22 dwisiswant0/apkleaks
23 MobSF/Mobile-Security-Framework-MobSF
24 ajinabraham/NodeJsScan
25 linkedin/qark
26 SonarSource/sonarqube
27 vincentcox/StaCoAn
28 SUPERAndroidAnalyzer/super
29 aquasecurity/trivy
30 webarx-security/wpbullet
31 archerysec/archerysec
32 softrams/bulwark
33 TFNS/CTFNote
34 lmco/dart
35 DefectDojo/django-DefectDojo
36 dradis/dradis-ce
37 evait-security/envizon
38 infobyte/faraday
39 GhostManager/Ghostwriter
40 deantonious/hackox
41 KvasirSe

In [5]:
git_api_url

NameError: name 'git_api_url' is not defined

In [236]:
# Retrieve Releases 
for i in [*range(0,df.shape[0])]:
    repo_id = df[df.index==i]['git_url'].values[0][19:]
    git_api_url = "https://api.github.com/repos/" + repo_id +"/releases" 
    #print(git_api_url)
    res = request('GET',url=git_api_url,auth=('wazaahhh', token))
    if res.status_code == 200 and len(res.text)>2 :
        k=0
        J = res.json()
        f = open(root_dir + "data/" + re.sub("/","_",repo_id) + "_releases.json","w")
        f.write(json.dumps(J))
        f.close()

        print(i,repo_id)
        for j in J:
            if len(j['assets']) >0 :
                k+=1
        print(k)

KeyboardInterrupt: 

In [32]:
repo_id = df[df.index==29]['git_url'].values[0][19:]
git_api_url = "https://api.github.com/repos/" + repo_id +"/releases" 
res = request('GET',url=git_api_url,auth=('wazaahhh', token))
if res.status_code == 200:
    J = res.json()
    f = open(root_dir "data/" + re.sub("/","_",repo_id) + "_releases.json","w")

In [53]:
for i,asset in enumerate(J[0]['assets']):
    print(i,J[0]['id'],asset['id'],asset['name'],asset['content_type'],asset['size'],asset['created_at'],asset['updated_at'],asset['download_count'])

0 64127345 62238658 bom.json application/json 193662 2022-04-11T16:29:58Z 2022-04-11T16:29:58Z 0
1 64127345 62238688 trivy_0.25.4_checksums.txt text/plain; charset=utf-8 1830 2022-04-11T16:30:07Z 2022-04-11T16:30:07Z 2669
2 64127345 62238663 trivy_0.25.4_FreeBSD-32bit.tar.gz application/gzip 15581340 2022-04-11T16:29:59Z 2022-04-11T16:30:00Z 0
3 64127345 62238662 trivy_0.25.4_FreeBSD-64bit.tar.gz application/gzip 16358824 2022-04-11T16:29:59Z 2022-04-11T16:30:00Z 3
4 64127345 62238675 trivy_0.25.4_Linux-32bit.deb application/vnd.debian.binary-package 15620394 2022-04-11T16:30:03Z 2022-04-11T16:30:04Z 4
5 64127345 62238685 trivy_0.25.4_Linux-32bit.rpm application/x-rpm 15586239 2022-04-11T16:30:06Z 2022-04-11T16:30:06Z 0
6 64127345 62238660 trivy_0.25.4_Linux-32bit.tar.gz application/gzip 15588594 2022-04-11T16:29:58Z 2022-04-11T16:29:59Z 2
7 64127345 62238683 trivy_0.25.4_Linux-64bit.deb application/vnd.debian.binary-package 16403442 2022-04-11T16:30:05Z 2022-04-11T16:30:05Z 290
8 6412

In [111]:
res = request('GET',url='https://api.github.com/repos/aquasecurity/trivy/contributors',auth=('wazaahhh', token))

In [312]:
def query_authed(github_api_url):
    res = request('GET',url=github_api_url,auth=('wazaahhh', token))
    if res.status_code == 200:
        J = res.json()

    elif res.status_code == 401:
        print(res,"... trying without auth token")
        res = request('GET',url=github_api_url)
        
        if res.status_code == 200:
            print(res)
            J = res.json()
    else:
        print(res)
    return J


def query_repo(github_api_url):
    
    J = []
    p=1
    try:
        j = query_authed(github_api_url + '?page=%s'%p)
        J = j
    except:
        return
        
    while len(j)>=30:
        print(p)
        j = query_authed(github_api_url + '?page=%s'%p)
        J = J + j
        p+=1
    
    return J


def query_user_commits(github_api_url):
    #github_api_url should of the format:  'https://api.github.com/search/commits?q=author:<username>'
    J = []
    p=1
    
    try:
        j = query_authed(github_api_url + '&page=%s'%p)['items']
        J = j
    except:
        return
        
    while len(j)>=30:
        
        j = query_authed(github_api_url + '&page=%s'%p)['items']
        J = J + j
        print(p,len(j),len(J))
        p+=1
    
    return J

In [306]:
p =3
github_api_url = 'https://api.github.com/search/commits?q=author:gblomqvist'
j = query_authed(github_api_url + '&page=%s'%p)
len(j),len(j['items'])

(3, 30)

In [258]:
#retrieve Commits

for i in [*range(545,df.shape[0])]:
    repo_id = df[df.index==i]['git_url'].values[0][19:]
    print(i,repo_id)
    
    J = query_repo(github_api_url)
    
    f = open(root_dir + "data/" + re.sub("/","_",repo_id) + "_commits.json","w")
    f.write(json.dumps(J))
    f.close()

545 doyensec/inql
546 projectdiscovery/interactsh
547 Lavakumar/IronWASP
548 jaeles-project/jaeles
549 mikehacksthings/jast
550 zigoo0/JSONBee
551 brendan-rius/c-jwt-cracker
552 lmammino/jwt-cracker
553 hahwul/jwt-hack
554 ticarpi/jwt_tool
555 AresS31/jwtcat
556 mzfr/liffy
557 OsandaMalith/LFiFreak/
<Response [404]>
558 D35m0nd142/LFISuite
559 lightbulb-framework/lightbulb-framework
560 P0cL4bs/Kadimus
561 .net/projects/malzilla/
<Response [404]>
562 mitmproxy/mitmproxy
563 Kong/mockbin
564 RedTeamPentesting/monsoon
565 mBouamama/MyJWT
566 sullo/nikto
567 codingo/NoSQLMap
568 torque59/Nosql-Exploitation-Framework
569 projectdiscovery/nuclei
570 TheBinitGhimire/NtHiM
571 rezasp/joomscan
572 zaproxy/zaproxy
573 BuffaloWill/oxml_xxe
574 lightos/Panoptic
575 roglew/pappy-proxy
576 devanshbatham/ParamSpider
577 .net/projects/paros/
<Response [404]>
578 rg/LaNMaSteR53/peepingtom/
<Response [404]>
579 ambionics/phpggc
580 NetSPI/PowerUpSQL
581 noraj/rabid
582 Runscope/requestbin
<Response [40

In [20]:
i = 115

In [37]:
repo_id = "zzzteph/weakpass"
p=1
github_api_url = 'https://api.github.com/repos/' + repo_id + '/commits?page=%s'%p
print(github_api_url)
j = query_authed('https://api.github.com/repos/' + repo_id + '/commits?page=%s'%p)

https://api.github.com/repos/zzzteph/weakpass/commits?page=1
<Response [401]>


UnboundLocalError: local variable 'J' referenced before assignment

## Retrieve Missing Repos

In [94]:
#open(root_dir + "missingcommits.txt",'r').read()

fileName=open(root_dir + "missingcommits.txt")
lines = [i for i in fileName.readlines()]

missing = []

for l in lines:
    l = re.sub('_','/',l[27:-14],1)
    missing.append(l)

In [133]:
for i,repo_id in enumerate(missing[103:]):
    print(i,repo_id)
    
    github_api_url = 'https://api.github.com/repos/' + repo_id + '/commits'
    
    J = query_repo(github_api_url)
    
    f = open(root_dir + "data/" + re.sub("/","_",repo_id) + "_commits.json","w")
    f.write(json.dumps(J))
    f.close()

0 Synacktiv/Modmobmap
1 sysdream/ligolo
2 Tak31337/medusa-gui
3 talsoft/vuldash
4 TarlogicSecurity/Arecibo
5 TarlogicSecurity/Chankro
6 teknogeek/ssrf-sheriff
7 TheCyberViking/Facebook_OSINT_Dump
8 TheCyberViking/PublicIntelligenceTool
9 thewhiteh4t/nexfil
10 tismayil/fockcache
11 torque59/Garfield
12 vegabird/prithvi
13 vegabird/xvna
14 VirtueSecurity/aws-extender
15 VirtueSecurity/aws-extender-cli
16 whitel1st/docem
17 whoot/Typo3Scan
18 woj-ciech/LeakLooker-X
19 xoreaxeaxeax/sandsifter
20 yeswehack/PwnFox
21 yeswehack/YesWeBurp
22 zigoo0/JSONBee
23 zzzteph/appsec.study
24 zzzteph/weakpass


## Retrieve Usernames

In [245]:
list_json = os.listdir(root_dir + "data/")

Authors = []

with_author = 0
no_author = 0

for file in list_json[:]:
    if file[-12:] == 'commits.json':
        J =json.loads(open(root_dir + "data/" + file).read())
        for commit in J:
            try:
                Authors.append(commit['author']['login'])
                with_author += 1
            except:
                no_author += 1
                continue

print(no_author/with_author)
Authors =  np.unique(Authors)
file = open(root_dir + "authors_usernames.txt",'w')

In [250]:
file = open(root_dir + "authors_usernames.txt",'w')
file.write("\n".join(Authors))
file.close()

In [286]:
len(Authors)

9279

## Retrieve User Events 

In [325]:
skipped_users = []

for username in Authors[30:100]:
    print(username)
    github_api_url = 'https://api.github.com/search/commits?q=author:%s'%username
    try:
        J = query_user_commits(github_api_url)
        f = open(root_dir + "data/users/commits_" + username + ".json","w")
        f.write(json.dumps(J))
        f.close()
    except:
        print("skipped : ", username)
        skipped_users.append(username)
        continue

0x6263
<Response [403]>
0x6368
<Response [403]>
0x6d696368
<Response [403]>
0x6d6f7468
<Response [403]>
0x71rex
<Response [403]>
0x8008135
<Response [403]>
0x90
<Response [403]>
0x9060
<Response [403]>
0xABD
<Response [403]>
0xAL3INO
<Response [403]>
0xD503
<Response [403]>
0xDezzy
<Response [403]>
0xFayden
<Response [403]>
0xGilda
<Response [403]>
0xHJK
<Response [403]>
0xINT3
<Response [403]>
0xInfection
<Response [403]>
0xItx
<Response [403]>
0xMcX
<Response [403]>
0xPanic
<Response [403]>
0xRet
<Response [403]>
0xShoreditch
<Response [403]>
0xSobky
0xTowel
<Response [403]>
0xXA
<Response [403]>
0xadam
<Response [403]>
0xbadjuju
<Response [403]>
0xbc
<Response [403]>
0xbsec
<Response [403]>
0xc0da
<Response [403]>
0xced
<Response [403]>
0xcpu
<Response [403]>
0xcpy
<Response [403]>
0xd0ug
<Response [403]>
0xddom
<Response [403]>
0xdevalias
<Response [403]>
0xdf0xdf
<Response [403]>
0xe7
<Response [403]>
0xflotus
<Response [403]>
0xiso
<Response [403]>
0xmostafam
<Response [403]>
0xr

In [251]:
fileName=open(root_dir + "authors_usernames.txt")
lines = [i[:-1] for i in fileName.readlines()]

In [266]:
github_api_url = 'https://api.github.com/search/commits?q=author:gblomqvist&sort=author-date&order=desc&page=1'
J = query_authed(github_api_url)
print(J)

{'total_count': 93, 'incomplete_results': False, 'items': [{'url': 'https://api.github.com/repos/Gallopsled/pwntools/commits/a5c0809f38a1dcbb9ed8cf9e37a42316236636d7', 'sha': 'a5c0809f38a1dcbb9ed8cf9e37a42316236636d7', 'node_id': 'MDY6Q29tbWl0OTc1MDIzMzphNWMwODA5ZjM4YTFkY2JiOWVkOGNmOWUzN2E0MjMxNjIzNjYzNmQ3', 'html_url': 'https://github.com/Gallopsled/pwntools/commit/a5c0809f38a1dcbb9ed8cf9e37a42316236636d7', 'comments_url': 'https://api.github.com/repos/Gallopsled/pwntools/commits/a5c0809f38a1dcbb9ed8cf9e37a42316236636d7/comments', 'commit': {'url': 'https://api.github.com/repos/Gallopsled/pwntools/git/commits/a5c0809f38a1dcbb9ed8cf9e37a42316236636d7', 'author': {'date': '2022-08-08T08:39:02.000-07:00', 'name': 'Gustaf Blomqvist', 'email': 'gustaf.blomqvist98@gmail.com'}, 'committer': {'date': '2022-08-08T23:00:30.000+02:00', 'name': 'Arusekk', 'email': 'arek_koz@o2.pl'}, 'message': 'Do not skip chars after non-printable characters\n\nCloses #2094', 'tree': {'url': 'https://api.github.

In [254]:
github_api_url = 'https://api.github.com/search/commits?q=author:gblomqvist&sort=author-date&order=desc'
J = query_repo(github_api_url)

In [263]:
github_api_url = 'https://api.github.com/search/commits?q=author:gblomqvist&sort=author-date&order=desc&page=4'
query_authed(github_api_url)

{'total_count': 93,
 'incomplete_results': False,
 'items': [{'url': 'https://api.github.com/repos/gblomqvist/blender-Wirebomb/commits/6eea235deaaf3163ac66a374eb300f704870986c',
   'sha': '6eea235deaaf3163ac66a374eb300f704870986c',
   'node_id': 'MDY6Q29tbWl0MzAwODM3NzM6NmVlYTIzNWRlYWFmMzE2M2FjNjZhMzc0ZWIzMDBmNzA0ODcwOTg2Yw==',
   'html_url': 'https://github.com/gblomqvist/blender-Wirebomb/commit/6eea235deaaf3163ac66a374eb300f704870986c',
   'comments_url': 'https://api.github.com/repos/gblomqvist/blender-Wirebomb/commits/6eea235deaaf3163ac66a374eb300f704870986c/comments',
   'commit': {'url': 'https://api.github.com/repos/gblomqvist/blender-Wirebomb/git/commits/6eea235deaaf3163ac66a374eb300f704870986c',
    'author': {'date': '2015-01-31T02:14:10.000+01:00',
     'name': 'GBlomqvist',
     'email': 'gustaf.blomqvist98@gmail.com'},
    'committer': {'date': '2015-01-31T02:14:10.000+01:00',
     'name': 'GBlomqvist',
     'email': 'gustaf.blomqvist98@gmail.com'},
    'message': 'test',


In [320]:
username = "gblomqvist"
github_api_url = 'https://api.github.com/search/commits?q=author:%s'%username
J = query_user_commits(github_api_url)

1 30 60
2 30 90
3 30 120
4 3 123
